In [6]:
import sys
import os as os
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import copy
import h5py
import keras as keras
from keras.layers import Input, merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dropout, Activation
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
import time
sys.path.append(os.path.abspath('squeezeNet_classification1'))
from squeezenet import *

In [7]:
## this code creates the ground truth images, and the corresponding heatMaps, aquired by hand labelling
## of the training data. 
## the input to my net will be an image of 720x1080x3
## the output is a heatMap of 16x27

projectPath = 'C:\\Users\\David\\Desktop\\GitFolder\\project1'
dataPath = 'C:\\Users\\David\\Desktop\\Fish\\train\\train'
scorePath = os.path.join('C:\\Users\\David\\Desktop\\Fish', 'train', 'binaryScoreMaps')

# run through binaryScoreMap folders and count how many are there
fishTypes = os.listdir(scorePath)
numOfImages = 0
for fishIdx in np.arange(len(fishTypes)):
    print "fishTypes : " + fishTypes[fishIdx]
    fishPath = os.path.join(scorePath, fishTypes[fishIdx])
    # filter all non jpg files\folders
    imageNames = [fn for fn in os.listdir(fishPath) if fn.endswith("jpg")]
    numOfImages = numOfImages + len(imageNames)
    
print 'number of score maps : ', numOfImages
h_score = 16 #720
h_img = 720
w_score = 27 #1080
w_img = 1080



fishTypes : ALB
fishTypes : BET
fishTypes : DOL
fishTypes : LAG
fishTypes : OTHER
fishTypes : SHARK
fishTypes : YFT
number of score maps :  3312


In [11]:
# create an array of all scoreMaps & all images
# scoreMaps are 4D and now contains also the number of classes
scoreMaps = np.zeros((len(fishTypes)+1, h_score, w_score, numOfImages)) # '+1' is to add a 'noFish' class
images = np.zeros((numOfImages,3, h_img, w_img), dtype=np.float32)
theNumberOne = np.ones((1,1,1,1))

# run through the data, and save all of it within an array
counter = 0
for fishIdx in np.arange(len(fishTypes)):
    fishPath = os.path.join(scorePath, fishTypes[fishIdx])
    # filter all non jpg files\folders
    scoreMapNames = [fn for fn in os.listdir(fishPath) if fn.endswith("jpg")]
    # run through the images and perform 2 things:
    # 1. resize the score images to be of size h_score x w_score
    # 2. perform all required data manipulation on the images themselfs..
    # save all...
    print 'fishType number : ' + np.str(fishIdx) + '/' + np.str(len(fishTypes))
    for imageIdx in np.arange(len(scoreMapNames)):
        # print the number of image
        #print 'image number : ' + np.str(counter) + '/' + np.str(numOfImages)
        # everything about the image
        imagePath = os.path.join(dataPath, fishTypes[fishIdx], scoreMapNames[imageIdx])
        img = misc.imread(imagePath).astype(np.float32)
        img = misc.imresize(img, (720, 1080)).astype(np.float32)
        #plt.imshow(img)        
        aux = copy.copy(img)
        img[:, :, 0] = aux[:, :, 2]
        img[:, :, 2] = aux[:, :, 0]
        # Remove train image mean
        img[:, :, 0] -= 104.006
        img[:, :, 1] -= 116.669
        img[:, :, 2] -= 122.679
        img = np.transpose(img, (2, 0, 1))
        img = np.expand_dims(img, axis=0)
        images[counter,:,:,:] = img
        
        #everything about the scoreMap
        scoreMapPath = os.path.join(fishPath, scoreMapNames[imageIdx])
        scoreMap = misc.imread(scoreMapPath, mode='L').astype(np.float32)
        scoreMap = misc.imresize(scoreMap, (h_score,w_score)).astype(np.bool)
        scoreMaps[fishIdx,:,:,counter] = scoreMap
        scoreMaps[len(fishTypes),:,:,counter] = theNumberOne-scoreMap
        counter += 1
        

fishType number : 0/7
fishType number : 1/7
fishType number : 2/7
fishType number : 3/7
fishType number : 4/7
fishType number : 5/7
fishType number : 6/7


In [12]:
# try training the network
model = get_squeezenet(len(fishTypes)+1,720,1080, dim_ordering='th')
model.compile(loss="categorical_crossentropy", optimizer="adam")
model.load_weights(os.path.abspath('squeezeNet_classification1/model/squeezenet_weights_th_dim_ordering_th_kernels.h5'), by_name=True)

transposedScoreMaps = np.transpose(scoreMaps, (3, 0, 1,2))
print 'ready to start training...'

ready to start training...


In [14]:

partialPath="weights-improvement-{epoch:02d}-{acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(os.path.join('C:\Users\David\Desktop\Fish\classification',partialPath), monitor='acc', verbose=1, save_best_only=True,save_weights_only = False, mode='max')
model.fit(x=images, y=transposedScoreMaps, batch_size=10, nb_epoch=4, verbose=1, callbacks=[checkpoint], validation_split=0.1, shuffle=True, class_weight=np.array([1.5, 0.5]), sample_weight=None, initial_epoch=0)
    #model.save('C:\Users\David\Desktop\Fish\kerasModel\SecondModel' + np.str(epochNum) + '.h5')

Train on 2980 samples, validate on 332 samples
Epoch 1/4
  40/2980 [..............................] - ETA: 5190s - loss: 12.6740

KeyboardInterrupt: 